# Letterboxd.com Recommendation System

## Business Case

In the following notebook, we aim to create various recommendation systems that will recommend 5 movies to any given Letterboxd user using various methods including non-personal recommendations, Surprise and PySpark. 



## SQL Database Diagram

Here is a quick look at the tables that are in the SQL database 'letterboxd.'

![db_diagram](db_diagram.png)

## Initial Examination of Data and EDA

In [ ]:
# import necessary libraries 
import pandas as pd
import plotly.express as px
import sqlite3
import numpy as np

In [2]:
# Intialize connection to the SQL database
conn = sqlite3.Connection('letterboxd.db')

In [ ]:
# Most rated films
highest_rated = """SELECT username, AVG(rating) as avg_rating
FROM ratings, films
WHERE ratings.film_id = films.lb_id
GROUP BY username
ORDER BY avg_rating DESC
"""

In [ ]:
user_avg_rating = pd.read_sql(highest_rated, conn)

user_avg_rating.head()

In [ ]:
fig = px.box(user_avg_rating, y="avg_rating", title='Average Rating Per User')
fig.show()


In [ ]:
# Drop any nulls and round to one decimal place
user_avg_rating = user_avg_rating.dropna().round(1)

In [ ]:
fig = px.histogram(user_avg_rating, x="avg_rating", title='Average Rating Per User')
fig.show()


## Non-Personalized Recommendations

In [ ]:
# Most popular, i.e. most rated

In [ ]:
# Most rated films
most_rated = """SELECT film_name, AVG(rating) as avg_rating, COUNT(film_id) as count
FROM ratings, films
WHERE ratings.film_id = films.lb_id
GROUP BY film_id
ORDER BY count DESC
"""

In [ ]:
top_20_most_rated = pd.read_sql(most_rated, conn)
top_20_most_rated = top_20_most_rated[:20]
top_20_most_rated

In [ ]:
fig = px.line(top_20_most_rated, x='film_name', y='count', title='20 Most Rated Films')
fig.show()

In [ ]:
top_20_most_rated_sort = top_20_most_rated.sort_values(['avg_rating'], ascending=False)
top_20_most_rated_sort

In [ ]:
fig = px.line(top_20_most_rated_sort, x='film_name', y='avg_rating', 
              title='20 Most Rated Films Avergage Rating')
fig.show()

Here we see that of the 20 most rated films Parasite, Spirited Away, The Shining, Pulp Fiction, Eternal Sunshine of the Spotless Mind are the highest rated films. 

## Collaborative model

In [ ]:
#user-user vs item-item

In [ ]:
rating = """SELECT *
FROM ratings"""

In [ ]:
users_ratings = pd.read_sql(rating, conn, index_col = 'index')
users_ratings.dropna(inplace=True)
print(users_ratings.info())
users_ratings.head()

In [ ]:
# Order of columns username, film_id, rating
users_ratings = users_ratings[['username', 'film_id', 'rating']]
users_ratings.head()

### Import Libraries, Train/Test Split

In [ ]:
# import libraries
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import knns, KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson

In [ ]:
reader = Reader()
data = Dataset.load_from_df(users_ratings,reader)

In [ ]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

In [ ]:
# train-test split
train, test = train_test_split(data, test_size=.2)

In [ ]:
train

### SVD

In [ ]:
svd = SVD()
svd.fit(train)
predictions = svd.test(test)

In [ ]:
accuracy.rmse(predictions)

In [ ]:
# Test Pediction
uid = 'ingloriousbasta'
iid = 346746

# get a prediction for specific users and items.
pred = svd.predict(uid, iid, verbose=True)

In [ ]:
# Test Pediction
uid = 'ingloriousbasta'
iid = 517828

# get a prediction for specific users and items.
pred = svd.predict(uid, iid, verbose=True)

### KNN

In [ ]:
sim_cos = {'name':'cosine', 'user_based':False}

In [ ]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(train)

In [ ]:
basic.sim

In [ ]:
predictions = basic.test(test)

In [ ]:
print(accuracy.rmse(predictions))

In [ ]:
sim_pearson = {'name':'pearson', 'user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(train)
predictions = basic_pearson.test(test)
print(accuracy.rmse(predictions))

In [ ]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

In [ ]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_baseline = knns.KNNBaseline(sim_options=sim_pearson)
knn_baseline.fit(trainset)
predictions = knn_baseline.test(testset)
print(accuracy.rmse(predictions))

### Alternating Least Square With PySpark

In [3]:
# import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext

spark = SparkSession\
        .builder\
        .appName('recsystem').config('spark.driver.host', 'localhost')\
        .getOrCreate()

In [4]:
db_path = 'letterboxd.db'
query = 'SELECT * from ratings'

conn = sqlite3.connect(db_path)
df = pd.read_sql_query(query, conn)

# Assign id numbers to usernames
df = df.assign(id=(df['username'].astype('category').cat.codes))



In [9]:
# Remove any films that have fewer than 100 ratings
threshold = 100 # Anything that occurs less than this will be removed.
value_counts = df['film_id'].value_counts() # Specific column 
to_remove = value_counts[value_counts <= threshold].index
df['film_id'].replace(to_remove, np.nan, inplace=True)

In [10]:
df.dropna(inplace=True)

In [11]:
df['film_id'] = df['film_id'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237964 entries, 4 to 904861
Data columns (total 5 columns):
index       237964 non-null int64
film_id     237964 non-null int64
username    237964 non-null object
rating      237964 non-null float64
id          237964 non-null int16
dtypes: float64(1), int16(1), int64(2), object(1)
memory usage: 9.5+ MB


In [12]:
df['film_id'].value_counts()

426406    768
406775    607
475370    583
326279    568
353117    550
459564    546
51444     523
404266    519
397859    512
95113     507
34722     495
460830    495
51432     491
422682    490
251943    485
240344    484
433863    482
444600    478
51921     474
149857    468
51568     468
51896     466
41352     463
259441    455
114564    453
424348    453
333029    451
312205    450
438511    448
171384    447
         ... 
40276     102
45263     102
205114    102
43015     102
15077     102
51472     102
69453     102
46084     102
318913    102
19921     102
47062     102
353273    102
266628    102
635253    102
47345     102
46778     102
174195    101
153285    101
50953     101
44920     101
46263     101
47913     101
49886     101
51163     101
345176    101
276291    101
51470     101
50224     101
50390     101
422900    101
Name: film_id, Length: 1237, dtype: int64

In [13]:
movie_ratings = spark.createDataFrame(data=df)

In [14]:
movie_ratings.dtypes

[('index', 'bigint'),
 ('film_id', 'bigint'),
 ('username', 'string'),
 ('rating', 'double'),
 ('id', 'bigint')]

In [15]:
movie_ratings = movie_ratings.drop('index')

In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# split into training and testing sets
(train, test) = movie_ratings.randomSplit([0.8, 0.2])

In [17]:
train

DataFrame[film_id: bigint, username: string, rating: double, id: bigint]

In [18]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,rank=4, regParam=0.1, userCol='id', itemCol='film_id', ratingCol='rating',
          coldStartStrategy='drop')

In [19]:
# fit the ALS model to the training set
model = als.fit(train)

In [20]:
# importing appropriate library
from pyspark.ml.evaluation import RegressionEvaluator

In [21]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')

In [22]:
predictions

DataFrame[film_id: bigint, username: string, rating: double, id: bigint, prediction: float]

In [23]:
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 0.7441200416067888


In [24]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# initialize the ALS model
als_model = ALS(userCol='id', itemCol='film_id', 
                ratingCol='rating', coldStartStrategy='drop')

# create the parameter grid                 
params = ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001, 0.1])\
          .addGrid(als_model.rank, [4, 10, 50]).build()


# instantiating crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(movie_ratings)    

# We see the best model has a rank of 50, so we will use that in our future models with this dataset
best_model.bestModel.rank

50

In [25]:
als = ALS(maxIter=5,rank=50, regParam=0.1, userCol='id', itemCol='film_id', ratingCol='rating',
          coldStartStrategy='drop')

# fit the ALS model to the training set
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 0.7153673049046974


In [29]:
db_path = 'letterboxd.db'
query = 'SELECT * from films'

conn = sqlite3.connect(db_path)
df_films = pd.read_sql_query(query, conn)

movie_titles = spark.createDataFrame(data=df_films)

In [30]:
movie_titles.head(5)

[Row(index=0, film_name='Parasite', lb_id=426406, lb_link='https://letterboxd.com//film/parasite-2019/', tmdb_id=496243, movie_tv='movie', release_year='2019', director='Bong Joon-ho'),
 Row(index=1, film_name='Joker', lb_id=406775, lb_link='https://letterboxd.com//film/joker-2019/', tmdb_id=475557, movie_tv='movie', release_year='2019', director='Todd Phillips'),
 Row(index=2, film_name='Knives Out', lb_id=475370, lb_link='https://letterboxd.com//film/knives-out-2019/', tmdb_id=546554, movie_tv='movie', release_year='2019', director='Rian Johnson'),
 Row(index=3, film_name='Pulp Fiction', lb_id=51444, lb_link='https://letterboxd.com//film/pulp-fiction/', tmdb_id=680, movie_tv='movie', release_year='1994', director='Quentin Tarantino'),
 Row(index=4, film_name='Inception', lb_id=34722, lb_link='https://letterboxd.com//film/inception/', tmdb_id=27205, movie_tv='movie', release_year='2010', director='Christopher Nolan')]

In [31]:
def username_retriever(user_id, rating_df):
    return rating_df.where(rating_df.id == user_id).take(1)[0]['username']

In [32]:
def id_retriever(username, rating_df):
    return rating_df.where(rating_df.username == username).take(1)[0]['id']

In [38]:
def name_retriever(movie_id, movie_title_df):
    return movie_title_df.where(movie_title_df.lb_id == movie_id).take(1)[0]['film_name']

In [34]:
users = movie_ratings.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs = model.recommendForUserSubset(users, 10)
recs = userSubsetRecs.take(1)

In [39]:
# use indexing to obtain the movie id of top predicted rated item
first_recommendation = recs[0]['recommendations'][0][0]

# use the name retriever function to get the values
name_retriever(first_recommendation, movie_titles)

'12 Angry Men'

In [42]:
def first_five_rec(username):
    user_id = id_retriever(username, movie_ratings)
    recommendations = model.recommendForAllUsers(5)
    first_five = recommendations.where(recommendations.id == user_id).collect()
    temp = first_five[0]
    temp = temp[1]
    
    print(f'First 5 Recommendations for {username}: \n')
    for x in range(len(temp)):
        print(f'{name_retriever(temp[x][0], movie_titles)} | predicted score = {round(temp[x][1], 2)}')

In [43]:
first_five_rec('4dollarshrimp')

First 5 Recommendations for 4dollarshrimp: 

Parasite | predicted score = 4.55
12 Angry Men | predicted score = 4.53
Stop Making Sense | predicted score = 4.53
Harakiri | predicted score = 4.51
Paris Is Burning | predicted score = 4.49


# Future Work:

- Dashboard?
- Website?
- Content-based recommendation